In [1]:
import numpy as np
import os
import sys
import pandas as pd
import pyreadr
from plotnine import *
from mizani.formatters import percent_format

In [2]:
sids = ['Anguilla' ,'American Samoa' ,'Jamaica' ,'Papua New Guinea' ,'British Virgin Islands' ,
        'Kiribati' ,'Saint Vincent and the Grenadines' ,'Vanuatu' ,'Bahamas' ,
        'Fiji' ,'Puerto Rico' ,'Timor-Leste' ,'Dominican Republic' ,'New Caledonia' ,
        'United States Virgin Islands' ,'Comoros' ,'Aruba' ,'Federated States of Micronesia' ,'Netherlands Antilles' ,
        'Solomon Islands' ,'Dominica' ,'Nauru' ,'Trinidad and Tobago' ,'Cape Verde' ,
        'Belize' ,'Guam' ,'Saint Lucia' ,'Tuvalu' ,'Guyana' ,
        'Northern Mariana Islands' ,'Seychelles' ,'Maldives' ,'Antigua and Barbuda' ,'Cook Islands' ,
        'Montserrat' ,'Samoa' ,'Cuba' ,'Marshall Islands' ,'Suriname' ,
        'Bahrain' ,'Barbados' ,'French Polynesia' ,'Saint Kitts and Nevis' ,'Tonga' ,
        'Grenada' ,'Niue' ,'Sao Tome and Principe' ,'Guinea-Bissau' ,'Haiti' ,'Palau' ,'Singapore' ,'Mauritius']
tier1_countries = ["USA", "CHN"]
tier1_countries_eu = ["Austria", "Belgium", "Bulgaria", "Croatia",  "Cyprus", 
                      "Czechia","Denmark", "Estonia", "Finland", "France","Germany","Greece", 
                      "Hungary", "Ireland", "Italy","Latvia", "Lithuania",
                      "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal","Romania", 
                      "Slovakia", "Slovenia","Spain", "Sweden", "United Kingdom"]
EEAandCO = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", 
            "Czechia","Denmark", "Estonia", "Finland", "France","Germany","Greece", 
            "Hungary", "Iceland","Ireland", "Italy","Latvia", "Liechtenstein", "Lithuania", 
            "Luxembourg", "Malta", "Netherlands", "Norway", "Poland", "Portugal","Romania", 
            "Slovakia", "Slovenia","Spain", "Sweden", "Switzerland", "Turkey", 
            "Albania",  "Bosnia and Herzegovina", "Kosovo", "Montenegro", "North Macedonia", "Serbia", 
            "United Kingdom"]
tier2_countries = ["POL","AUS", "ZAF", "BRA", "IND", "IDN", "DEU","FRA","GBR"] # 

In [3]:
grouping_2023 = pd.read_excel("Country Names and groupings - 2023 Report.xlsx", engine='openpyxl')
grouping_2023.set_axis(grouping_2023.iloc[0], axis=1, inplace=True)
grouping_2023 = grouping_2023.iloc[1:]
grouping_2023.reset_index(drop = True, inplace = True)
grouping_2023 = grouping_2023.iloc[:, :5]
grouping_2023 = grouping_2023.loc[grouping_2023.isna().sum(axis = 1) < 5]
grouping_2023["Country Name to use"].replace({"Republic of Korea": "Korea, Republic of",
            "Russian Federation":"Russian Federation", "Federated States of Micronesia":"Micronesia",
            'Bolivia': "Bolivia", "Democratic Republic of the Congo": 'Congo, Democratic Republic of the',
            "United Republic of Tanzania": "Tanzania, United Republic of",
            "Islamic Republic of Iran": "Iran, Islamic Republic of", "United States of America":"United States",
            "Republic of Moldova": "Moldova, Republic of", "Vietnam":"Viet Nam", "Guinea Bissau":"Guinea-Bissau",
            "Bolivarian Republic of Venezuela": "Venezuela","Hong Kong SAR (China)":"Hong Kong, China (SAR)",
            "Macao SAR (China)":"Macao, China (SAR)", "Occupied Palestinian territory": "Palestine, State of", 
            "Côte d'Ivoire": "Cote d'Ivoire"}, inplace = True)

In [4]:
WHO_Region_2021 = grouping_2023.groupby("WHO Region").agg({"Country Name to use":lambda x: list(x)}).reset_index()
WHO_Region_2021.set_index("WHO Region", inplace = True)

In [5]:
LC_grouping = grouping_2023.groupby("LC Grouping").agg({"Country Name to use":lambda x: list(x)}).reset_index()
LC_grouping.set_index("LC Grouping", inplace = True)

In [6]:
hdi = pd.read_csv("2020_hdi.csv", header = None)
hdi.set_axis(["index", "Country", "score", "HDI"], axis=1, inplace=True)
hdi.Country.replace({"Korea (Republic of)": "Korea, Republic of", "Russian Federation":"Russian Federation",
            'Bolivia (Plurinational State of)': "Bolivia", "Congo (Democratic Republic of the)": 'Congo, Democratic Republic of the',
            "Tanzania (United Republic of)": "Tanzania, United Republic of",
            "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
            "Moldova (Republic of)": "Moldova, Republic of", "Micronesia (Federated States of)": "Micronesia", 
                     "Eswatini (Kingdom of)":"Eswatini", 
            "Venezuela (Bolivarian Republic of)": "Venezuela", 
            "Côte d'Ivoire": "Cote d'Ivoire"}, inplace = True)
hdi = pd.merge(hdi, grouping_2023[["Country Name to use", "HDI Level (2021)"]], left_on = "Country", right_on = "Country Name to use", how = "outer")
hdi[hdi.isna().sum(axis = 1)<5].reset_index(inplace = True, drop = True)

In [7]:
##### load data
folder_kwdata = "Multiglobal_result_2023/" 

df_health = pd.read_pickle(folder_kwdata + 'df_kwic_health.pkl')
df_climate = pd.read_pickle(folder_kwdata + 'df_kwic_climate.pkl')
df_intersection = pd.read_csv(folder_kwdata + 'df_kwic_health_climate.csv')
df_gender = pd.read_csv(folder_kwdata + 'gender.csv')
df_covid = pd.read_csv(folder_kwdata + 'covid.csv')

In [9]:
csr = pd.read_csv('globaldata_2023/multiglobal_english_text.csv', chunksize = 10000)
#if folder == "EU_result_2023/":
 #   EU = pd.DataFrame()
  #  for df in csr:
   #     csr_mask = [True if i in EEAandCO else False for i in df.Country]
    #    EU = EU.append(df[csr_mask])
    #csr = EU
#elif folder == "English_2023/":
 #   EN = pd.DataFrame()
  #  for df in csr:
   #     EN = EN.append(df[df.English >0])
   # csr = EN
#elif (folder == "Multiglobal_result_2023/") or (folder == "country_result_2023/"):
glob = pd.DataFrame()
for df in csr:
    glob = glob.append(df)
csr = glob

In [10]:
dd = csr[['Participant', 'Sector', 'Country', 'Year', 'Id', 'code', "English"]]

In [11]:
csr = dd.merge(hdi[['Country Name to use', "HDI Level (2021)"]], left_on = "Country", right_on = "Country Name to use",how = "left")

In [12]:
csr.Country = csr.Country.apply(lambda x: x.replace(".", ","))

In [13]:
def sids_merg(row):
    sids_sign = None
    if row["Country"] in sids:
        sids_sign = "SIDS"
    elif (row["Country"] in tier1_countries_eu) or (row["code"] in tier1_countries):
        sids_sign = "Tier 1"
    elif row["code"] in tier2_countries:
        sids_sign = "Tier 2"
    else: 
        print(row["Country"])
    return sids_sign
csr["SIDS"] = csr.apply(lambda x: sids_merg(x), axis = 1)

Japan
Nepal
Japan
Switzerland
Israel
Japan
Nepal
Nepal
Korea, Republic of
Japan
Japan
United Arab Emirates
Bolivia
Korea, Republic of
Turkey
Canada
Japan
Canada
Saudi Arabia
Switzerland
Japan
Japan
Korea, Republic of
Ghana
Guatemala
Nigeria
Switzerland
Japan
Japan
Kuwait
Mongolia
Japan
Korea, Republic of
Norway
Japan
Malaysia
Japan
Argentina
United Arab Emirates
Turkey
Mexico
Turkey
Japan
Paraguay
Egypt
Russian Federation
Paraguay
Norway
Chile
Japan
Kenya
Japan
Japan
Japan
Switzerland
Japan
Korea, Republic of
Kazakhstan
Korea, Republic of
Japan
Turkey
Egypt
Kenya
Switzerland
Jordan
United Arab Emirates
Japan
Japan
Kenya
South Sudan
Norway
Sri Lanka
Nepal
Japan
Canada
Japan
Turkey
United Arab Emirates
Japan
Switzerland
Japan
Japan
Bermuda
Japan
Japan
Mexico
Iceland
Thailand
Mexico
Norway
Pakistan
Egypt
Korea, Republic of
Thailand
Sri Lanka
Viet Nam
Pakistan
Japan
Korea, Republic of
Japan
Thailand
Uganda
Kenya
Egypt
Switzerland
Japan
Japan
Japan
Japan
Norway
Chile
United Arab Emirates
Ja

Sri Lanka
Norway
Mexico
Israel
Korea, Republic of
Canada
Monaco
Egypt
Pakistan
Nigeria
Japan
Pakistan
Mexico
Canada
Norway
Switzerland
Colombia
Belarus
Norway
Kenya
Japan
Korea, Republic of
Korea, Republic of
Colombia
Russian Federation
Egypt
Nigeria
Norway
United Arab Emirates
Ukraine
Korea, Republic of
Korea, Republic of
New Zealand
Myanmar
Switzerland
Switzerland
Sri Lanka
Lebanon
Korea, Republic of
Viet Nam
Korea, Republic of
Japan
Ukraine
Norway
Korea, Republic of
Norway
Japan
Chile
United Arab Emirates
Japan
Japan
Korea, Republic of
Colombia
Argentina
Japan
Canada
Korea, Republic of
Japan
Norway
Kenya
Kenya
Russian Federation
United Arab Emirates
Kenya
Kenya
Japan
Egypt
Turkey
Korea, Republic of
Colombia
Nigeria
Sri Lanka
Korea, Republic of
Korea, Republic of
Uganda
Argentina
Malaysia
Lebanon
Lebanon
Georgia
Korea, Republic of
Nepal
Bangladesh
Japan
Ukraine
Japan
Thailand
Egypt
Canada
Korea, Republic of
Switzerland
Kenya
Myanmar
Japan
Nigeria
Kenya
Myanmar
Bangladesh
Colombia
Can

Japan
Pakistan
Argentina
Japan
Tunisia
Japan
Colombia
Switzerland
Lebanon
Sri Lanka
Paraguay
Japan
Namibia
Mexico
Panama
Mexico
Mexico
Iceland
Argentina
Norway
Japan
Sri Lanka
Thailand
Sri Lanka
Japan
Bangladesh
Lebanon
Sri Lanka
Japan
Mexico
Lebanon
Egypt
Myanmar
Colombia
Switzerland
Turkey
Lebanon
Uganda
Uganda
Tanzania, United Republic of
Norway
Armenia
Japan
Lebanon
Iran, Islamic Republic of
Ghana
Japan
Japan
Lebanon
Egypt
Japan
Israel
United Arab Emirates
Bangladesh
Sri Lanka
Kenya
United Arab Emirates
Japan
United Arab Emirates
Bangladesh
Japan
Japan
Switzerland
Japan
Japan
North Macedonia
Paraguay
Myanmar
Colombia
Korea, Republic of
Kenya
Liechtenstein
Oman
Mexico
Mexico
Lebanon
Japan
Norway
Japan
Mexico
Palestine, State of
Lebanon
Belarus
Egypt
Japan
Switzerland
United Arab Emirates
Kenya
Ecuador
Japan
Mexico
Myanmar
Switzerland
Myanmar
Japan
Japan
Japan
Egypt
United Arab Emirates
Korea, Republic of
Lebanon
Lebanon
Argentina
Japan
Serbia
Kenya
Norway
Norway
Japan
Sri Lanka
Turk

Bangladesh
Japan
Kenya
Lebanon
Myanmar
Mexico
Kenya
Pakistan
Canada
Turkey
Kenya
Myanmar
Mexico
Switzerland
Mexico
Ghana
Switzerland
Chile
Liechtenstein
Canada
Norway
Switzerland
Belarus
Sri Lanka
Malaysia
Sudan
Mexico
Mexico
Egypt
Lebanon
Kenya
Georgia
Israel
Mexico
Myanmar
Switzerland
Pakistan
Egypt
Myanmar
Peru
Switzerland
Norway
Sri Lanka
Korea, Republic of
Chile
Norway
Kenya
Kenya
Switzerland
Bangladesh
Thailand
Colombia
Sri Lanka
Malaysia
Kenya
Mozambique
Switzerland
Malaysia
Lebanon
Nigeria
Egypt
Thailand
Japan
Egypt
Norway
Turkey
Nigeria
Bangladesh
Iraq
Lebanon
Myanmar
Turkey
Canada
Turkey
United Arab Emirates
Norway
Chile
Switzerland
Egypt
Bangladesh
Qatar
Philippines
Colombia
Canada
Switzerland
Norway
Malaysia
Myanmar
Sri Lanka
Mexico
United Arab Emirates
Canada
Canada
Azerbaijan
Lebanon
Oman
Japan
Myanmar
Canada
Norway
Uganda
Norway
Norway
Japan
Ukraine
Argentina
Switzerland
Switzerland
Switzerland
Mexico
Peru
Iraq
Pakistan
Bangladesh
Mexico
Syrian Arab Republic
Switzerland


North Macedonia
Nigeria
Bangladesh
Norway
Thailand
Pakistan
Kuwait
Egypt
Canada
Switzerland
Japan
Thailand
Japan
Mexico
Norway
Korea, Republic of
Colombia
Norway
Canada
Korea, Republic of
Mexico
Syrian Arab Republic
Colombia
Norway
Sri Lanka
Philippines
Korea, Republic of
Turkey
Lebanon
Turkey
Switzerland
Japan
Argentina
Colombia
Mexico
Bangladesh
Nigeria
Korea, Republic of
Turkey
Mexico
Bangladesh
Sri Lanka
Egypt
Japan
Ghana
Russian Federation
Russian Federation
Russian Federation
Switzerland
Lebanon
United Arab Emirates
Korea, Republic of
Colombia
Egypt
Egypt
Korea, Republic of
Costa Rica
Sri Lanka
Argentina
Kenya
Switzerland
Japan
Switzerland
Lebanon
Thailand
United Arab Emirates
Nigeria
Korea, Republic of
Bangladesh
Kazakhstan
Japan
Norway
Nepal
Jordan
Philippines
Turkey
Viet Nam
Turkey
Ukraine
Switzerland
Switzerland
Japan
Korea, Republic of
Mexico
Turkey
Kenya
Sri Lanka
Kenya
Switzerland
Iceland
Myanmar
Sri Lanka
Iceland
Korea, Republic of
Switzerland
Switzerland
Korea, Republic 

...

Costa Rica
Panama
Panama
Argentina
Colombia
Peru
Ecuador
Paraguay
Ecuador
Chile
Bolivia
Colombia
Argentina
Colombia
Uruguay
Colombia
Colombia
Paraguay
Paraguay
Mexico
Colombia
Mexico
Colombia
Paraguay
Colombia
Colombia
Peru
Peru
Colombia
Argentina
Mexico
Paraguay
Paraguay
Mexico
Colombia
Paraguay
Colombia
Colombia
Bolivia
Chile
Ecuador
Argentina
Ecuador
Argentina
Colombia
Panama
Peru
Peru
Peru
Mexico
Chile
Ecuador
Chile
Argentina
Argentina
Argentina
Panama
Argentina
Paraguay
Mexico
Panama
Panama
Panama
Costa Rica
Mexico
Mexico
Mexico
Mexico
Costa Rica
Venezuela
Colombia
Paraguay
Uruguay
Peru
Colombia
Colombia
Colombia
Peru
Panama
Colombia
Colombia
Chile
Argentina
Peru
Colombia
Panama
Argentina
Mexico
Paraguay
Argentina
Colombia
Uruguay
Argentina
Argentina
Panama
Colombia
Bolivia
Chile
Peru
Colombia
Costa Rica
Colombia
Mexico
Mexico
Mexico
Panama
Colombia
Colombia
Colombia
Colombia
Colombia
Uruguay
Argentina
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Argentina
Colombia
Colomb

Colombia
Mexico
Colombia
Mexico
Mexico
Mexico
Colombia
Ecuador
Panama
Panama
Mexico
Peru
Mexico
Mexico
Colombia
Peru
Mexico
Mexico
Colombia
Peru
Nicaragua
Colombia
Argentina
Argentina
Argentina
Colombia
Mexico
Mexico
Mexico
Uruguay
Mexico
Mexico
Ecuador
Argentina
Ecuador
Panama
Colombia
Mexico
Mexico
Argentina
Colombia
Chile
Argentina
Argentina
Ecuador
Mexico
Argentina
Morocco
Mexico
Mexico
Mexico
Mexico
Colombia
Panama
Panama
Panama
Argentina
Colombia
Colombia
Mexico
Colombia
Mexico
Colombia
Colombia
Ecuador
Mexico
Panama
Argentina
Argentina
Colombia
Colombia
Mexico
Mexico
Peru
Colombia
Argentina
Bolivia
Argentina
Colombia
Mexico
Mexico
Peru
Uruguay
Mexico
Mexico
Colombia
Colombia
Nicaragua
Colombia
Mexico
Argentina
Argentina
Morocco
Costa Rica
Peru
Mexico
Mexico
Peru
Mexico
Mexico
Mexico
Mexico
Argentina
Mexico
Costa Rica
Argentina
Ecuador
Mexico
Mexico
Colombia
Mexico
Colombia
Uruguay
Mexico
Argentina
Panama
Colombia
Colombia
Mexico
Mexico
Argentina
Peru
Chile
Colombia
Colombia
Colo

Mexico
Paraguay
Chile
Colombia
Colombia
Colombia
Colombia
Paraguay
Uruguay
Peru
Colombia
Colombia
Mexico
Ecuador
Argentina
Chile
Colombia
Peru
Argentina
Paraguay
Colombia
Colombia
Ecuador
Panama
Mexico
Ecuador
Colombia
Panama
Mexico
Chile
Colombia
Peru
Mexico
Argentina
Peru
Colombia
Colombia
Colombia
Ecuador
Chile
Mexico
Colombia
Mexico
Mexico
Mexico
Panama
Guatemala
Bolivia
Bolivia
Argentina
Colombia
Chile
Mexico
Colombia
Colombia
Mexico
Costa Rica
Argentina
Peru
Panama
Argentina
Colombia
Argentina
Argentina
Argentina
Argentina
Argentina
Mexico
Colombia
Colombia
Argentina
Colombia
Colombia
Paraguay
Mexico
Canada
Colombia
Peru
Mexico
Argentina
Mexico
Bolivia
Argentina
Nicaragua
Ecuador
Peru
Nicaragua
Mexico
Argentina
Argentina
Chile
Mexico
Argentina
Colombia
Mexico
Colombia
Colombia
Chile
Mexico
Costa Rica
Chile
Ecuador
Chile
Mexico
Paraguay
Mexico
Chile
Mexico
Colombia
Mexico
Argentina
Panama
Bolivia
Mexico
Argentina
Mexico
Panama
Peru
Panama
Colombia
Uruguay
Ecuador
Mexico
Mexico
Col

Paraguay
Colombia
Argentina
Colombia
Chile
Mexico
Guatemala
Mexico
Colombia
Colombia
Colombia
Colombia
Mexico
Mexico
Mexico
Mexico
Mexico
Mexico
Colombia
Colombia
Mexico
Uruguay
Colombia
Argentina
Chile
Mexico
Mexico
Mexico
Colombia
Mexico
Argentina
Colombia
El Salvador
Peru
Costa Rica
Peru
Peru
Argentina
Colombia
Mexico
Mexico
Argentina
Colombia
Colombia
Colombia
Ecuador
Colombia
Argentina
Chile
Mexico
Chile
Colombia
Mexico
Colombia
Paraguay
Mexico
Costa Rica
Argentina
Mexico
Costa Rica
Colombia
Mexico
Mexico
Peru
Argentina
Argentina
Argentina
Panama
Colombia
Honduras
Mexico
Mexico
Peru
Mexico
Mexico
Argentina
Colombia
Mexico
Chile
Colombia
Argentina
Ecuador
Ecuador
Argentina
Chile
Mexico
Mexico
Colombia
Mexico
Mexico
Colombia
Argentina
Ecuador
Mexico
Mexico
Venezuela
Mexico
Mexico
Colombia
Chile
Colombia
Argentina
El Salvador
Mexico
Colombia
Colombia
Mexico
Mexico
Mexico
Mexico
Paraguay
Peru
Mexico
Colombia
Colombia
Bolivia
Peru
Mexico
Colombia
Mexico
Peru
Colombia
Ecuador
Colombia
E

Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japan
Japa

In [14]:
def who_seatch(x):
    region = None
    for k, v in WHO_Region_2021.iterrows():
        if x in v.tolist()[0]:
            region = k 
    if not region:
        print(x)
    return region
csr["WHO"] = csr.Country.apply(who_seatch)

Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Liechtenstein
Cape Verde
Cape Verde
Cape Verde
Liechtenstein


In [15]:
def LC_seatch(x):
    region = None
    for k, v in LC_grouping.iterrows():
        if x in v.tolist()[0]:
            region = k 
    if not region:
        print(x)
    return region
csr["LC_grouping"] = csr.Country.apply(LC_seatch)

Cape Verde
Cape Verde
Cape Verde


In [16]:
def agg_func(df, key = "intersection"):
    agg_inter = pd.DataFrame(df.groupby("Id").count().key_word).reset_index()
    agg_inter["Id"] = agg_inter["Id"].astype("int")
    agg_inter.rename(columns = {"key_word": key}, inplace = True)
    return agg_inter

In [17]:
csr = csr[csr.Year > 2010]
print(csr.shape)
reference_all = pd.merge(csr[["Participant",'Sector', 'Year', 'Id', "code", "Country", "WHO", "HDI Level (2021)", "SIDS", "LC_grouping", "English"]], agg_func(df_intersection, "intersection"), on = "Id", how = "left")
reference_all = pd.merge(reference_all, agg_func(df_health, "health"), on = "Id", how = "left")
reference_all = pd.merge(reference_all, agg_func(df_climate, "climate"), on = "Id", how = "left")
reference_all = pd.merge(reference_all, agg_func(df_gender, "gender"), on = "Id", how = "left")
reference_all = pd.merge(reference_all, agg_func(df_covid, "covid"), on = "Id", how = "left")
reference_all = reference_all.fillna(0)
#reference["n_company_year"] = reference.groupby(["Year"]).Id.transform("nunique")
reference_all["total_texts"] = reference_all.groupby(["Year", "Sector"]).Id.transform("nunique")

(45449, 12)


In [20]:
import warnings
warnings.filterwarnings("ignore")
print(reference_all.shape)
# multi_global
folder = "Multiglobal_result_2023/" 
path = "results_2023/" + folder
try: 
    os.mkdir(path)
except: pass
plotting(reference_all, path = path) 

(45449, 17)


In [21]:
# Europe
csr_mask = [True if i in EEAandCO else False for i in reference_all.Country]
referece_EU = reference_all[csr_mask]
print(referece_EU.shape)
folder = "EU_result_2023/" # "country_result_2023/"#"EU_result_2023/"  #  #  "English_2023/" # 
path = "results_2023/" + folder
try: 
    os.mkdir(path)
except: pass
plotting(referece_EU, path = path) 

(25272, 22)


In [22]:
# English
referece_English = reference_all[reference_all.English >0]
folder = "English_2023/"
path = "results_2023/" + folder
print(referece_English.shape)
try: 
    os.mkdir(path)
except: pass
plotting(referece_English, path = path) 

(24360, 22)


In [26]:
# run country specific analysis 
Country_lst = reference_all.Country.unique()
path_ = "results_2023/country_result_2023/"
done = []#["Kuwait", "Montenegro", "Uzbekistan", "Morocco", "Ghana"]
Country_lst = list(set(Country_lst).difference(set(done)))
print(Country_lst)

['Indonesia', 'Poland', 'Iraq', 'Egypt', 'Somalia', 'Ecuador', 'Afghanistan', 'South Africa', 'Philippines', 'Tunisia', 'Senegal', 'Denmark', 'Serbia', 'Estonia', 'Nepal', 'Syrian Arab Republic', 'Belgium', 'Bosnia and Herzegovina', 'Gambia', 'Czechia', 'Lithuania', 'Bermuda', 'Thailand', 'United Arab Emirates', 'Switzerland', 'Iceland', 'China', 'Peru', 'Nigeria', 'Honduras', 'Bolivia', "Cote d'Ivoire", 'Slovakia', 'Albania', 'Montenegro', 'Tanzania, United Republic of', 'Sierra Leone', 'Viet Nam', 'Kuwait', 'United Kingdom', 'Burundi', 'Togo', 'Malta', 'Luxembourg', 'Netherlands', 'Chile', 'Sudan', 'Japan', 'Colombia', 'Oman', 'Kazakhstan', 'Italy', 'Austria', 'Uruguay', 'Trinidad and Tobago', 'Bulgaria', 'Bangladesh', 'North Macedonia', 'Papua New Guinea', 'Norway', 'Armenia', 'Malawi', 'Iran, Islamic Republic of', 'Myanmar', 'Cape Verde', 'Chad', 'Russian Federation', 'Bahrain', 'Sao Tome and Principe', 'Uzbekistan', 'Slovenia', 'Kenya', 'South Sudan', 'Azerbaijan', 'Argentina', 'C

In [ ]:
# Country Specific generation
import warnings
warnings.filterwarnings("ignore")
print(reference_all.shape)
for Country in Country_lst:
    print(Country)
    path = path_ + Country +"/" 
    try:
        os.mkdir(path)   
    except:
        pass
    reference = reference_all[reference_all.Country == Country]
    print(reference.shape)
    plotting(reference, path = path)

(45449, 22)
Indonesia
(226, 22)
Poland
(213, 22)
Iraq
(44, 22)
Egypt
(230, 22)
Somalia
(4, 22)
Ecuador
(401, 22)
Afghanistan
(2, 22)
South Africa
(251, 22)
Philippines
(65, 22)
Tunisia
(120, 22)
Senegal
(11, 22)
Denmark
(2326, 22)
Serbia
(119, 22)
Estonia
(14, 22)
Nepal
(42, 22)
Syrian Arab Republic
(80, 22)
Belgium
(261, 22)
Bosnia and Herzegovina
(7, 22)
Gambia
(2, 22)
Czechia
(28, 22)
Lithuania
(180, 22)
Bermuda
(20, 22)
Thailand
(152, 22)
United Arab Emirates
(267, 22)
Switzerland
(681, 22)
Iceland
(87, 22)
China
(639, 22)
Peru
(455, 22)
Nigeria
(172, 22)
Honduras
(17, 22)
Bolivia
(84, 22)
Cote d'Ivoire
(5, 22)
Slovakia
(30, 22)


In [19]:
def plotting(reference, path = path):
    reference["intersection_texts"] = reference.groupby(["Year", "Sector", "Participant", "Id"]).intersection.transform("sum") > 0
    reference["climate_texts"] = reference.groupby(["Year", "Sector", "Participant", "Id"]).climate.transform("sum") > 0
    reference["health_texts"] = reference.groupby(["Year", "Sector", "Participant", "Id"]).health.transform("sum") > 0
    avg_sector = reference[reference.Year == 2022].groupby("Sector").agg({"intersection":"mean", "health":"mean", "climate":"mean",}).reset_index()
    avg_sector = avg_sector.round(2).sort_values("Sector")
    sum_1 = reference.agg({"Participant": ["count", "nunique",],}).T.reset_index(drop = True)
    sum_2 = reference.agg({ "health_texts": ["sum"], "climate_texts": "sum", "intersection_texts": "sum", }).reset_index(drop = True)
    summary = pd.concat([sum_1, sum_2, pd.DataFrame(sum_2.squeeze()/sum_1["count"].values).T], axis=1, )
    summary = summary.round({"health_texts":2,"climate_texts": 2, "intersection_texts": 3})
    summary.set_axis(["Companies (N)", "Companies (Unique)", "Health, (N)", "Climate, (N)", "Intersection, (N)", 
                      "Health, %", "Climate, %", "Intersection, %"], axis=1, inplace=True)
    # 0_0_data_summary
    summary.to_csv(path + '0_0_data_summary.csv', index = False)

    sum_1 = reference.groupby(["Year"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "Companies (N)"], axis=1, inplace=True)
    sum_2 = reference.groupby(["Year"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", }).reset_index(drop = True)
    sum_2.set_axis(["Climate (N)", "Health (N)", "Intersection (N)"], axis=1, inplace=True)
    sum_3 = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Companies (N)"]).round(2))
    sum_3.set_axis(["Climate (Prop)", "Health (Prop)", "Intersection (Prop)"], axis=1, inplace=True)
    yearly_breakdown = pd.concat([sum_1, sum_2, sum_3], axis=1, )
    yearly_breakdown.to_csv(path + '0_1_yearly_breakdown.csv', index = False)

    reference_health = reference[reference.Sector == "Health Care Equipment & Ser..."]
    sum_1 = reference_health.groupby(["Year"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "Companies (N)"], axis=1, inplace=True)
    sum_2 = reference_health.groupby(["Year"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", }).reset_index(drop = True)
    sum_2.set_axis(["Climate (N)", "Health (N)", "Intersection (N)"], axis=1, inplace=True)
    sum_3 = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Companies (N)"]).round(2))
    sum_3.set_axis(["Climate (Prop)", "Health (Prop)", "Intersection (Prop)"], axis=1, inplace=True)
    yearly_breakdown_health = pd.concat([sum_1, sum_2, sum_3], axis=1, )
    
    try:
    # WHO
        sum_1 = reference.groupby(["Year", "WHO"]).agg({"Participant": "count",}).reset_index()
        sum_1.set_axis(["Year", "WHO", "Total documents"], axis=1, inplace=True)
        sum_2 = reference.groupby(["Year", "WHO"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
        sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
        who_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
        who_prop = who_prop[who_prop.WHO != 0].sort_values(["WHO", "Year"])

        sum_1 = reference_health.groupby(["Year", "WHO"]).agg({"Participant": "count",}).reset_index()
        sum_1.set_axis(["Year", "WHO", "Total documents"], axis=1, inplace=True)
        sum_2 = reference_health.groupby(["Year", "WHO"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
        sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
        who_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
        who_prop_health = who_prop_health[who_prop_health.WHO != 0].sort_values(["WHO", "Year"])
    except:pass

    # SIDS
    try:
        sum_1 = reference.groupby(["Year", "SIDS"]).agg({"Participant": "count",}).reset_index()
        sum_1.set_axis(["Year", "SIDS", "Total documents"], axis=1, inplace=True)
        sum_2 = reference.groupby(["Year", "SIDS"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
        sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
        sids_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
        sids_prop = sids_prop[sids_prop.SIDS != 0].sort_values(["SIDS", "Year"])

        sum_1 = reference_health.groupby(["Year", "SIDS"]).agg({"Participant": "count",}).reset_index()
        sum_1.set_axis(["Year", "SIDS", "Total documents"], axis=1, inplace=True)
        sum_2 = reference_health.groupby(["Year", "SIDS"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
        sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
        sids_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
        sids_prop_health = sids_prop_health[sids_prop_health.SIDS != 0].sort_values(["SIDS", "Year"])
    except:pass

    # LC grouping
    sum_1 = reference.groupby(["Year", "LC_grouping"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "LC_grouping", "Total documents"], axis=1, inplace=True)
    sum_2 = reference.groupby(["Year", "LC_grouping"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
    sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
    LCGrouping_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
    LCGrouping_prop = LCGrouping_prop[LCGrouping_prop.LC_grouping != 0].sort_values(["LC_grouping", "Year"])

    sum_1 = reference_health.groupby(["Year", "LC_grouping"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "LC_grouping", "Total documents"], axis=1, inplace=True)
    sum_2 = reference_health.groupby(["Year", "LC_grouping"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
    sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
    LCGrouping_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
    LCGrouping_prop_health = LCGrouping_prop_health[LCGrouping_prop_health.LC_grouping != 0].sort_values(["LC_grouping", "Year"])

    # HDI
    sum_1 = reference.groupby(["Year", "HDI Level (2021)"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "HDI Level (2021)", "Total documents"], axis=1, inplace=True)
    sum_2 = reference.groupby(["Year", "HDI Level (2021)"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
    sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
    hdi_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
    hdi_prop = hdi_prop[hdi_prop["HDI Level (2021)"] != 0].sort_values(["HDI Level (2021)", "Year"])

    sum_1 = reference_health.groupby(["Year", "HDI Level (2021)"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Year", "HDI Level (2021)", "Total documents"], axis=1, inplace=True)
    sum_2 = reference_health.groupby(["Year", "HDI Level (2021)"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
    sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
    hdi_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
    hdi_prop_health = hdi_prop_health[hdi_prop_health["HDI Level (2021)"] != 0].sort_values(["HDI Level (2021)", "Year"])

    sum_1 = reference.groupby(["Sector"]).agg({"Participant": "count",}).reset_index()
    sum_1.set_axis(["Sector", "Total documents"], axis=1, inplace=True)
    sum_2 = reference.groupby(["Sector"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
    sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
    sector_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
    sector_prop = sector_prop.sort_values(["Sector"])

    reference_df_sector = reference.groupby(["Sector"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_who = reference.groupby(["WHO", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", })
    reference_who["sum"] = reference_who["intersection"] + reference_who["climate"] + reference_who["health"]
    reference_who.reset_index(inplace = True)
    reference_hdi = reference.groupby(["HDI Level (2021)", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_hdi.HDI = pd.Categorical(reference_hdi["HDI Level (2021)"], categories=['Low',
           'Medium','High',  'Very High'], ordered=True)
    reference_hdi.dropna(inplace = True)

    reference_hdi_health = reference_health.groupby(["HDI Level (2021)", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_hdi_health.HDI = pd.Categorical(reference_hdi_health["HDI Level (2021)"], categories=['Low',
           'Medium','High',  'Very High'], ordered=True)
    reference_hdi_health.dropna(inplace = True)

    reference_sids = reference.groupby(["SIDS", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_sids.SIDS = pd.Categorical(reference_sids.SIDS, categories=['SIDS',
           'Tier 1','Tier 2',], ordered=True)
    reference_sids.dropna(inplace = True)

    reference_sids_health = reference_health.groupby(["SIDS", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_sids_health.SIDS = pd.Categorical(reference_sids_health.SIDS, categories=['SIDS',
           'Tier 1','Tier 2',], ordered=True)
    reference_sids_health.dropna(inplace = True)



    reference_lcg = reference.groupby(["LC_grouping", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_lcg["sum"] = reference_lcg["intersection"] + reference_lcg["climate"] + reference_lcg["health"]
    reference_lcg.reset_index(inplace = True)
    reference_lcg.dropna(inplace = True)

    reference_lcg_health = reference_health.groupby(["LC_grouping", "Year"]).agg({"intersection": "sum",
                                               "climate": "sum",
                                               "health":"sum", }).reset_index()
    reference_lcg_health["sum"] = reference_lcg_health["intersection"] + reference_lcg_health["climate"] + reference_lcg_health["health"]
    reference_lcg_health.reset_index(inplace = True)
    reference_lcg_health.dropna(inplace = True)

    
    yearly_breakdown[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]].to_csv(path+ "1a_prop_of_companies.csv")
    plt = pd.melt(yearly_breakdown[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]], id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)", "Climate (Prop)", "Health (Prop)"], ordered=True)
    a1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable"), size=0.8) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--"),) + \
    annotate("text", x = 2013, y = 0.9,# 0.8,  
             label = "Health", color = "#619cff") + \
    annotate("text", x = 2014, y = 0.4,#0.4,
             label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2012, y = 0.05, 
             label = "Intersection", color = "red") + \
    scale_y_continuous(labels = percent_format(), limits = (0, 1)) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Proportion of companies, %") 
    a1.save(path +'1a_prop_of_companies.png')
    a1.save(path +'1a_prop_of_companies.pdf')


    yearly_breakdown_health[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]].to_csv(path+ "2p_prop_of_companies.csv")
    plt = pd.melt(yearly_breakdown_health[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]], id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)", "Climate (Prop)", "Health (Prop)"], ordered=True)
    p2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
    annotate("text", x = 2021, y = 0.95,# 0.8,  
             label = "Health", color = "#619cff") + \
    annotate("text", x = 2020, y = 0.5,#0.4,
             label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2021, y = 0.1, 
             label = "Intersection", color = "red") + \
    scale_y_continuous(labels = percent_format(), limits = (0, 1)) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Proportion of companies, %") 
    p2.save(path +'2p_prop_of_companies.png')
    p2.save(path +'2p_prop_of_companies.pdf')

    # breaks = (20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000)
    b1csv = reference.groupby("Year").agg({"intersection":"sum", "health":"sum", "climate":"sum"}).reset_index()
    b1csv.to_csv(path+ "1b_number_of_references.csv")
    plt = pd.melt(b1csv, id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
    b1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
    annotate("text", x = 2016, y = 50000, label = "Health", color = "#619cff") + \
    annotate("text", x = 2018, y = 9000, label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2013, y = 5000, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None), ) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Total number of references") 
    b1.save(path +'1b_number_of_references.png')
    b1.save(path +'1b_number_of_references.pdf')

    q2csv = reference_health.groupby("Year").agg({"intersection":"sum", "health":"sum", "climate":"sum"}).reset_index()
    q2csv.to_csv(path+ "2q_number_of_references.csv")
    plt = pd.melt(q2csv, id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
    q2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
    annotate("text", x = 2020, y =4000, label = "Health", color = "#619cff") + \
    annotate("text", x = 2016, y = 800, label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2018, y = 200, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None), ) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Total number of references") 
    # 1b_number_of_references
    q2.save(path +'2q_number_of_references.png')
    q2.save(path +'2q_number_of_references.pdf')

    c1csv = reference.groupby("Year").agg({"intersection":"sum"}).reset_index()
    c1csv.to_csv(path+ "1c_number_of_references_intersection.csv")
    plt = pd.melt(c1csv, id_vars=['Year'] )
    c1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    annotate("text", x = 2015, y = 2, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None),) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red",)) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Total number of references") 
    c1.save(path +'1c_number_of_references_intersection.png')
    c1.save(path +'1c_number_of_references_intersection.pdf')

    r2csv = reference_health.groupby("Year").agg({"intersection":"sum"}).reset_index()
    r2csv.to_csv(path+ "2r_number_of_references_intersection.csv")
    plt = pd.melt(r2csv, id_vars=['Year'] )
    r2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    annotate("text", x = 2020, y = 4, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None),) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red",)) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Total number of references") 
    r2.save(path +'2r_number_of_references_intersection.png')
    r2.save(path +'2r_number_of_references_intersection.pdf')

    yearly_breakdown[["Year", "Intersection (Prop)"]].to_csv(path+ "1d_proportion_intersection.csv")
    plt = pd.melt(yearly_breakdown[["Year", "Intersection (Prop)"]], id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)",], ordered=True)
    d1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    annotate("text", x = 2012, y = 0.1, label = "Intersection", color = "red") + \
    scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red",  )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Proportion of companies, %") 
    d1.save(path +'1d_proportion_intersection.png')
    d1.save(path +'1d_proportion_intersection.pdf')

    e1csv = reference.groupby("Year").agg({"intersection":"mean", "health":"mean", "climate":"mean"}).reset_index()
    e1csv.to_csv(path+ "1e_avg_references.csv")
    plt = pd.melt(e1csv, id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
    e1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
    annotate("text", x = 2015, y = 25, label = "Health", color = "#619cff") + \
    annotate("text", x = 2014, y = 8, label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2017, y = 2, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None), ) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Average number of references") 
    e1.save(path +'1e_avg_references.png')
    e1.save(path +'1e_avg_references.pdf')

    e1csv = reference_health.groupby("Year").agg({"intersection":"mean", "health":"mean", "climate":"mean"}).reset_index()
    e1csv.to_csv(path+ "2s_avg_references.csv")
    plt = pd.melt(e1csv, id_vars=['Year'] )
    plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
    e1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
    geom_path(aes(linetype = "variable")) + \
    scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
    annotate("text", x = 2018, y = 60, label = "Health", color = "#619cff") + \
    annotate("text", x = 2019, y = 20, label = "Climate Change", color = "darkgreen") + \
    annotate("text", x = 2021, y = 5, label = "Intersection", color = "red") + \
    scale_y_continuous(limits = (0, None), ) + \
    theme_minimal() + \
    theme(legend_position = "none") + \
    scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
    scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
    labs(y = "Average number of references") 
    e1.save(path +'2s_avg_references.png')
    e1.save(path +'2s_avg_references.pdf')
    
    try:
        reference_lcg[reference_lcg.LC_grouping !=0].to_csv(path+ "1q_LC-Grouping_number_references.csv")
        q1 = ggplot(reference_lcg[reference_lcg.LC_grouping !=0], aes(x = "Year", y = "intersection", color = "LC_grouping")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references") 
        q1.save(path +'1q_LC-Grouping_number_references.png')
        q1.save(path +'1q_LC-Grouping_number_references.pdf')

        LCGrouping_prop_health[LCGrouping_prop_health.LC_grouping !=0].to_csv(path+ "2q_LC-Grouping_prop_references.csv")
        q2 = ggplot(LCGrouping_prop_health[LCGrouping_prop_health.LC_grouping !=0], aes(x = "Year", y = "Intersection", color = "LC_grouping")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        q2.save(path +'2q_LC-Grouping_prop_references.png')
        q2.save(path +'2q_LC-Grouping_prop_references.pdf')

        LCGrouping_prop[LCGrouping_prop.LC_grouping !=0].to_csv(path+ "1q1_LC-grouping_prop_references.csv")
        q11 = ggplot(LCGrouping_prop[LCGrouping_prop.LC_grouping !=0], aes(x = "Year", y = "Intersection", color = "LC_grouping")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        q11.save(path +'1q1_LC-grouping_prop_references.png')
        q11.save(path +'1q1_LC-grouping_prop_references.pdf')
    except:pass
    
    try:
        reference_who[reference_who.WHO !=0].to_csv(path+ "1f_who_number_references.csv")
        f1 = ggplot(reference_who[reference_who.WHO !=0], aes(x = "Year", y = "intersection", color = "WHO")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references") 
        f1.save(path +'1f_who_number_references.png')
        f1.save(path +'1f_who_number_references.pdf')

        reference_who_health = reference_health.groupby(["WHO", "Year"]).agg({"intersection": "sum",
                                                   "climate": "sum",
                                                   "health":"sum", })
        reference_who_health["sum"] = reference_who_health["intersection"] + reference_who_health["climate"] + reference_who_health["health"]
        reference_who_health.reset_index(inplace = True)

        reference_who_health["Sector"] = "Health Care Equipment & Ser..."
        reference_who_health[reference_who_health.WHO !=0].to_csv(path+ "2t_who_number_references.csv")
        f1 = ggplot(reference_who_health[reference_who_health.WHO !=0], aes(x = "Year", y = "intersection", color = "WHO")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references") 
        f1.save(path +'2t_who_number_references.png')
        f1.save(path +'2t_who_number_references.pdf')

        who_prop[who_prop.WHO !=0].to_csv(path+ "1g_who_prop_references.csv")
        g1 = ggplot(who_prop[who_prop.WHO !=0], aes(x = "Year", y = "Intersection", color = "WHO")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        g1.save(path +'1g_who_prop_references.png')
        g1.save(path +'1g_who_prop_references.pdf')

        who_prop_health[who_prop_health.WHO !=0].to_csv(path+ "2u_who_prop_references.csv")
        g1 = ggplot(who_prop_health[who_prop_health.WHO !=0], aes(x = "Year", y = "Intersection", color = "WHO")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        g1.save(path +'2u_who_prop_references.png')
        g1.save(path +'2u_who_prop_references.pdf') 
    except:pass
    
    try:
        #1h_sids_number_references.csv
        reference_sids["sum"] = reference_sids["intersection"] + reference_sids["climate"] + reference_sids["health"] 
        reference_sids.to_csv(path+ "1h_sids_number_references.csv")
        h1 = ggplot(reference_sids[reference_sids.SIDS != 0], aes(x = "Year", y = "intersection", color = "SIDS")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references")
        h1.save(path +'1h_sids_number_references.png')
        h1.save(path +'1h_sids_number_references.pdf')

        #2v_sids_number_references.csv
        reference_sids_health["sum"] = reference_sids_health["intersection"] + reference_sids_health["climate"] + reference_sids_health["health"] 
        reference_sids.to_csv(path+ "2v_sids_number_references.csv")
        h1 = ggplot(reference_sids_health[reference_sids_health.SIDS != 0], aes(x = "Year", y = "intersection", color = "SIDS")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references")
        h1.save(path +'2v_sids_number_references.png')
        h1.save(path +'2v_sids_number_references.pdf')

        sids_prop[sids_prop.SIDS !=0].to_csv(path+ "1i_sids_prop_references.csv")
        i1 = ggplot(sids_prop[sids_prop.SIDS !=0], aes(x = "Year", y = "Intersection", color = "SIDS")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        i1.save(path +'1i_sids_prop_references.png')
        i1.save(path +'1i_sids_prop_references.pdf')

        sids_prop_health[sids_prop_health.SIDS !=0].to_csv(path+ "2w_sids_prop_references.csv")
        i1 = ggplot(sids_prop_health[sids_prop_health.SIDS !=0], aes(x = "Year", y = "Intersection", color = "SIDS")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        i1.save(path +'2w_sids_prop_references.png')
        i1.save(path +'2w_sids_prop_references.pdf')
    except:pass
    
    try:
        reference_hdi["sum"] = reference_hdi["intersection"] + reference_hdi["climate"] + reference_hdi["health"] 
        reference_hdi.to_csv(path+ "1j_hdi_number_references.csv")
        j1 = ggplot(reference_hdi[reference_hdi["HDI Level (2021)"] != 0], aes(x = "Year", y = "intersection", color = "HDI Level (2021)")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references")
        j1.save(path +'1j_hdi_number_references.png')
        j1.save(path +'1j_hdi_number_references.pdf')

        reference_hdi_health["sum"] = reference_hdi_health["intersection"] + reference_hdi_health["climate"] + reference_hdi_health["health"] 
        reference_hdi_health["Sector"] = "Health Care Equipment & Ser..."
        reference_hdi_health.to_csv(path+ "2x_hdi_number_references.csv")
        j1 = ggplot(reference_hdi_health[reference_hdi_health["HDI Level (2021)"] != 0], aes(x = "Year", y = "intersection", color = "HDI Level (2021)")) + \
        geom_path(aes()) + \
        scale_y_continuous(limits = (0, None), ) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Total number of references")
        j1.save(path +'2x_hdi_number_references.png')
        j1.save(path +'2x_hdi_number_references.pdf')

        hdi_prop[hdi_prop["HDI Level (2021)"] !=0].to_csv(path+ "1k_hdi_prop_references.csv")
        k1 = ggplot(hdi_prop[hdi_prop["HDI Level (2021)"] !=0], aes(x = "Year", y = "Intersection", color = "HDI Level (2021)")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, 0.5)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        # 1k_hdi_prop_references
        k1.save(path +'1k_hdi_prop_references.png')
        k1.save(path +'1k_hdi_prop_references.pdf')

        hdi_prop_health[hdi_prop_health["HDI Level (2021)"] !=0].to_csv(path+ "2y_hdi_prop_references.csv")
        k1 = ggplot(hdi_prop_health[hdi_prop_health["HDI Level (2021)"] !=0], aes(x = "Year", y = "Intersection", color = "HDI Level (2021)")) + \
        geom_path(aes()) + \
        scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
        theme_minimal() + \
        theme(legend_position = "right") + \
        scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
        labs(y = "Proportion of companies, %") 
        # 1k_hdi_prop_references
        k1.save(path +'2y_hdi_prop_references.png')
        k1.save(path +'2y_hdi_prop_references.pdf')
    except:pass

    # 1l_sector_references_2022
    reference_df_sector.to_csv(path+"1l_sector_references_2022.csv")

    # 1m_avg_sector_references_2020.csv
    avg_sector.to_csv(path + "1m_avg_sector_references_2022.csv")

    # 1n_prop_companies_sector.csv  2022
    sector_prop["Year"] = 2022
    sector_prop.to_csv(path + "1n_prop_companies_sector.csv")
    melt = sector_prop.melt(id_vars=["Sector", 'Year'])
    melt.variable = pd.Categorical(melt.variable, categories=["Intersection", "Climate", "Health"], ordered=True)
    n1 = ggplot(melt, aes(x = "Sector", y = "value", fill = "variable")) + \
      geom_bar(stat = "identity", position = "dodge") + \
      theme_minimal() + \
      scale_y_continuous(labels = percent_format()) + \
      theme(axis_text_x = element_text(angle = 90, vjust = 1, hjust=0.5, size = 10 ), ) + \
      theme(legend_position = (0.5, -0.4), legend_direction="horizontal", figure_size=(14, 7)) + \
      scale_fill_manual(values=("red", "darkgreen", "#619cff" )) +\
      labs(fill = "",
           y = "Proportion of companies, %",
           x = "\nSector")
    n1.save(path +'1n_prop_companies_sector.png')
    n1.save(path +'1n_prop_companies_sector.pdf')

    reference["covid_texts"] = reference.groupby(["Year", "Sector", "Id"]).covid.transform("sum") > 0
    reference["gender_texts"] = reference.groupby(["Year", "Sector", "Id"]).gender.transform("sum") > 0
    
    try:
        covid_inter = reference[(reference.Year > 2019) & (reference.intersection > 0)].groupby(["Year"]).agg({"Id": "nunique", "covid":"sum", "covid_texts": "sum",})
        covid_inter["prop"] = (covid_inter["covid_texts"]/covid_inter["Id"]).round(2)
        covid_inter.set_axis(["total_docs", "hits", "documents", "prop_doct"], axis=1, inplace=True)
        covid_inter["hits"] = covid_inter["hits"].astype("int")
        covid_inter.reset_index(inplace = True)
        covid_inter.to_csv(path + "1o_intersection_covid.csv")

        o1 = ggplot(covid_inter, aes(x = "Year", y = "prop_doct", group = 1)) +\
          geom_line(color = "#cc0055") +\
          theme_minimal() +\
          labs(x= "Year",
               y = "Proportion of documents\n Covid mention in intersection, %\n") +\
          scale_y_continuous(labels = percent_format(), limits = (0, None))
        o1.save(path +'1o_prop_intersecion_covid.png')
        o1.save(path +'1o_prop_intersecion_covid.pdf')
    except:pass
    
    try:
        # 1o_intersection_gender
        gender_inter = reference[(reference.Year > 2010) & (reference.intersection > 0)].groupby(["Year"]).agg({"Id": "nunique", "gender":"sum", "gender_texts": "sum", })
        gender_inter["prop"] = (gender_inter["gender_texts"]/gender_inter["Id"]).round(2)
        gender_inter.set_axis(["total_docs", "hits", "documents", "prop_doct"], axis=1, inplace=True)
        gender_inter["hits"] = gender_inter["hits"].astype("int")
        gender_inter.reset_index(inplace = True)

        gender_inter.to_csv(path + "1o_intersection_gender.csv", index = False)
        o1 = ggplot(gender_inter, aes(x = "Year", y = "prop_doct", group = 1)) +\
          geom_line(color = "#cc0055") +\
          theme_minimal() +\
          labs(x= "Year",
               y = "Proportion of documents\n Gender mention in intersection, %\n") +\
          scale_y_continuous(labels = percent_format(), limits = (0, None))
        o1.save(path +'1o_prop_intersecion_gender.png')
        o1.save(path +'1o_prop_intersecion_gender.pdf')
    except:pass